# The SQL Murder mystery

È avvenuto un crimine e il detective ha bisogno del tuo aiuto! Il detective ti ha dato il
rapporto sulla scena del crimine, ma in qualche modo l'hai perso. Ricordi vagamente quel delitto
è stato un **omicidio** avvenuto il **15 gennaio 2018** e avvenuto in **SQL city**. Inizia recuperando il corrispondente rapporto sulla scena del crimine dalla polizia database del dipartimento. Se vuoi ottenere il massimo da questo mistero, prova a lavorare
attraverso di esso solo utilizzando l'ambiente SQL e astenendosi dall'utilizzare un blocco note.

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%sql duckdb:///data/sql-murder-mystery.db


Connecting to 'duckdb:///the-sql-murder-mystery/sql-murder-mystery.db'

In [2]:
%config SqlMagic.autopandas = False
%config SqlMagic.feedback = True
%config SqlMagic.displaycon = False

![db schema](images/schema.png)

In [54]:
%%sql
a <<
SELECT *
FROM crime_scene_report
WHERE date = 20180115
AND type = 'murder'
AND city = 'SQL City';

In [55]:
a.DataFrame().to_latex("tables/query1.tex")
a

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [56]:
%%sql
a <<
SELECT *
FROM person
WHERE address_street_name LIKE '%Northwestern%'
ORDER BY address_number DESC
LIMIT 1;

In [57]:
a.DataFrame().to_latex("tables/query2.tex")
a

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [58]:
%%sql
a << 
SELECT *
FROM person
WHERE name LIKE '%Annabel%'
AND address_street_name LIKE 'Franklin Ave';

In [59]:
a.DataFrame().to_latex("tables/query3.tex")
a

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [60]:
%%sql
a <<
SELECT b.name AS name, a.transcript AS transcript
FROM interview AS a
INNER JOIN person AS b
ON a.person_id = b.id
WHERE person_id IN (14887, 16371);

In [61]:
a.DataFrame().to_latex("tables/query4.tex")
a

name,transcript
Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [62]:
%%sql 
a << 
SELECT b.name
FROM get_fit_now_check_in AS a
INNER JOIN get_fit_now_member AS b
ON a.membership_id = b.id
INNER JOIN person AS c
ON b.person_id = c.id
INNER JOIN drivers_license AS d
ON c.license_id = d.id
WHERE a.check_in_date = 20180109
AND d.plate_number LIKE '%H42W%'; --patente

In [63]:
a.DataFrame().to_latex("tables/query5.tex")
a

name
Jeremy Bowers


In [64]:
%%sql
a <<
SELECT value FROM solution;

In [65]:
a.DataFrame().to_latex("tables/query6.tex")
a

value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [66]:
%%sql 
a <<
SELECT b.name AS name, a.transcript AS transcript
FROM interview AS a
INNER JOIN person AS b
ON a.person_id = b.id
WHERE b.name = 'Jeremy Bowers'

In [67]:
a.DataFrame().to_latex("tables/query7.tex")
a

name,transcript
Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [68]:
%%sql 
a <<
SELECT a.id, a.name, COUNT(*) AS 'conteggio presenza'
FROM person AS a 
INNER JOIN income AS b
ON a.ssn = b.ssn
INNER JOIN drivers_license AS c
ON a.license_id = c.id
INNER JOIN facebook_event_checkin AS e
ON a.id = e.person_id
WHERE c.height BETWEEN 65 AND 67
AND c.hair_color = 'red'
AND c.car_make = 'Tesla'
AND c.car_model = 'Model S'
AND event_name LIKE '%SQL Symphony%'
AND date BETWEEN 20171201 AND 20171231
GROUP BY a.id, a.name; 

In [69]:
a.DataFrame().to_latex("tables/query8.tex")
a

id,name,conteggio presenza
99716,Miranda Priestly,3


In [27]:
%%sql

SELECT *
FROM interview 
WHERE person_id = 99716;

person_id,transcript
